##### This file contains all the functions and structures used for conducting experiments with the simple CNN networks
##### All of the functions are analogous to the functions contained in the ANN_CODE notebook, with some minor changes to allow for execution with the CNN architectures

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time

class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            # print(validation_loss,self.min_validation_loss,(self.min_validation_loss + self.min_delta) )
            if self.counter >= self.patience:
                return True
        return False
    
def train_data_mirror(X_train, y_train):
   
   '''This Function is used to perform mirroring data augmentation (use only for train data)'''

   titles = X_train.columns

   columns_titles = np.concatenate((titles[int((len(titles))/2):],titles[0:int((len(titles))/2)]))
   

   X_train_2 = X_train.reindex(columns=columns_titles) 

   X_train_2.columns = X_train.columns

   y_train_2 = 1-y_train
   X_train = pd.concat([X_train, X_train_2], axis=0)
   y_train = pd.concat([y_train, y_train_2], axis=0)

   return X_train, y_train


def create_data_loaders(data, batch_size, train_mirror=False, standarize=True):


    # Scale the data?
    if standarize:  
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]

        scaler = StandardScaler()

        X_ = scaler.fit_transform(X)

        data_ = pd.DataFrame(X_)
        data_['winner'] = y

        data = data_
    
    # Split data into training and testing sets
    train_data, test_data = train_test_split(data, test_size=0.15)
    train_data, validation_data = train_test_split(train_data, test_size=0.15/0.85)
 
    # Data augmentation flip?

    if train_mirror:
        
        X_train, y_train = train_data_mirror(train_data.iloc[:, :-1], train_data.iloc[:, -1])

    # Convert data to PyTorch tensors and create data loaders
    train_dataset = TensorDataset(torch.from_numpy(train_data.iloc[:,:-1].values.astype(np.float32)), torch.from_numpy(train_data.iloc[:, -1].values.reshape(-1, 1).astype(np.float32)))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    valid_dataset = TensorDataset(torch.from_numpy(validation_data.iloc[:,:-1].values.astype(np.float32)), torch.from_numpy(validation_data.iloc[:, -1].values.reshape(-1, 1).astype(np.float32)))
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    test_dataset = TensorDataset(torch.from_numpy(test_data.iloc[:, :-1].values.astype(np.float32)), torch.from_numpy(test_data.iloc[: ,-1].values.reshape(-1, 1).astype(np.float32)))
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    
    print(f'Data loaded -- starting model training...')

    return train_loader, valid_loader, test_loader

def train(model, loader, optimizer, criterion, device, mode, epoch, print_history = True):
    model.train()
    # Train the model for 1 epoch
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        batch_size = inputs.size(0)
        total += batch_size
        try:
            loss = criterion(outputs, labels)
        except:
            loss = criterion(outputs.squeeze(1), labels.squeeze(1))

        
        loss.mean().backward()
        optimizer.step()
        running_loss += loss.item() * batch_size / len(loader.dataset)
        correct += (predicted == labels).sum().item()
        
    accuracy =  correct/total
    if print_history:
        print(f"Epoch {epoch+1} {mode} loss: {running_loss}")
        print(f"Epoch {epoch+1} {mode} accuracy: {accuracy}")

    return running_loss, accuracy


def test(model, loader, criterion, device, mode, epoch, print_history = True):
    model.eval()
    # Evaluate the neural network on the testing set
    running_loss = 0.0
    avg_inference_time = 0
    counter = 0
    with torch.no_grad():
        correct = 0
        total = 0
        for data in loader:
            counter += 1

            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            start = time.time()
            outputs = model(inputs)
            end = time.time()
            predicted = (outputs > 0.5).float()
            batch_size = inputs.size(0)
            total += batch_size
            batch_inference_time = end-start
            avg_inference_time += batch_inference_time
            try:
                loss = criterion(outputs, labels)
            except:
                loss = criterion(outputs.squeeze(1), labels.squeeze(1))
            running_loss += loss.item() * batch_size / len(loader.dataset)
            correct += (predicted == labels).sum().item()

        accuracy =  correct / total
        
        if print_history:
            print(f"Epoch {epoch+1} {mode} loss: {running_loss}")
            print(f"Epoch {epoch+1} {mode} accuracy: {accuracy}")
        
    return running_loss, accuracy, avg_inference_time/counter


def plot_loss(train_loss, validation_loss, title):
    plt.grid(True)
    plt.xlabel("subsequent epochs",fontsize=16)
    plt.ylabel('average loss',fontsize=16)
    plt.plot(range(1, len(train_loss)+1), train_loss, 'o-', label='training')
    plt.plot(range(1, len(validation_loss)+1), validation_loss, 'o-', label='validation')
    plt.legend()
    plt.title(title,fontsize=20)
    plt.tick_params(labelsize=12)

    plt.show()
    
def plot_acc(train_loss, validation_loss, title):
    plt.grid(True)
    plt.xlabel("subsequent epochs",fontsize=16)
    plt.ylabel('average accuracy',fontsize=16)
    plt.plot(range(1, len(train_loss)+1), train_loss, 'o-', label='training')
    plt.plot(range(1, len(validation_loss)+1), validation_loss, 'o-', label='validation')
    plt.legend()
    plt.title(title,fontsize=20)
    plt.tick_params(labelsize=12)
    plt.show()

In [3]:
def go_CNN_model(data, model, optimizer, num_epochs, batch_size, device = 'cuda',train_mirror=False, early_stopping=False, es_patience = 5, es_delta = 0.01):
    
    hi_accuracy=0
    train_loss_arr = []
    validation_loss_arr = []

    train_acc_arr = []
    validation_acc_arr = []

    train_loader, validation_loader, test_loader = create_data_loaders(data, batch_size,train_mirror) # Initialize the dataloaders from dataset


    # Create CNN classifier model and optimizer
    model = model.to(device)
    criterion = nn.BCELoss()
    optimizer = optimizer

    # Train the model
    if(early_stopping):
        early_stopper = EarlyStopper(patience=es_patience, min_delta=es_delta) # Enable early stopping

    for epoch in range(num_epochs):
        train_loss, train_accuracy = train(model,train_loader, optimizer, criterion=criterion, device=device, mode='Train', epoch=epoch)
        valid_loss, valid_accuracy = test(model, validation_loader, criterion=criterion, device=device, mode='Validation', epoch=epoch)

        if(early_stopping):
            if early_stopper.early_stop(valid_loss):
                break

        
        

        train_loss_arr.append(train_loss)
        train_acc_arr.append(train_accuracy)

        

        validation_loss_arr.append(valid_loss)
        validation_acc_arr.append(valid_accuracy)

        try:
            if valid_accuracy>=hi_accuracy:
                hi_accuracy = valid_accuracy
                print(f'New best state saved with valid. acc. = {hi_accuracy}.')
                model_state = model.state_dict()
        except:
            print("Cannot yet save model state dict.")

        



    plot_loss(train_loss=train_loss_arr, validation_loss=validation_loss_arr, title='CNN model loss')
    plot_acc(train_acc_arr, validation_acc_arr, title='CNN model accuracy')

    test(model, test_loader, criterion=criterion, device=device, mode='Test', epoch=epoch)

    return model_state

In [4]:
def create_loader_cv(data, batch_size, train_mirror=False, standarize=True):


    # Scale the data?
    if standarize:  
        data.reset_index(drop=True, inplace=True)
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]

        scaler = StandardScaler()

        X_ = scaler.fit_transform(X)

        data_ = pd.DataFrame(X_)
        data_['winner'] = y

        data = data_
    
    # Split data into training and testing sets
    # train_data, test_data = train_test_split(data, test_size=0.15)
    # train_data, validation_data = train_test_split(train_data, test_size=0.15/0.85)
 
    # Data augmentation flip?

    if train_mirror:
        
        X, y = train_data_mirror(data.iloc[:, :-1], data.iloc[:, -1])

    # Convert data to PyTorch tensors and create data loaders
    data_dataset = TensorDataset(torch.from_numpy(data.iloc[:,:-1].values.astype(np.float32)), torch.from_numpy(data.iloc[:, -1].values.reshape(-1, 1).astype(np.float32)))
    data_loader = DataLoader(data_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    
    print(f'Data loaded succesfully!')

    return data_loader

In [5]:
def cv_CNN_model(train_loader, test_loader, model, criterion, optimizer, num_epochs, device='cuda', early_stopping = False, es_patience = 5, es_delta = 0.01, plot_history = True, print_history = True):
    
    model_state = None
    hi_accuracy = 0
    train_loss_arr = []
    validation_loss_arr = []

    train_acc_arr = []
    validation_acc_arr = []



    # train_loader, validation_loader, test_loader = create_data_loaders(data, batch_size,train_mirror) # Initialize the dataloaders from dataset

    model = model.to(device) # Initialize linear model with layer sizes
    # criterion = nn.BCELoss() # Initialize loss function - Binary Cross Entropy Loss
    optimizer = optimizer # Initialize optimizer with starting learning rate
    try:
        criterion = criterion()
    except:
        print('focal loss')

    # Initialize result collecting lists for later plotting
    train_loss_arr = []
    validation_loss_arr = []

    train_acc_arr = []
    validation_acc_arr = []

    avg_inference_time = 0

    # Train the model
    if(early_stopping):
        early_stopper = EarlyStopper(patience=es_patience, min_delta=es_delta) # Enable early stopping

    for epoch in range(num_epochs):

        train_loss, train_accuracy = train(model,train_loader, optimizer, criterion=criterion, device=device, mode='Train', epoch=epoch, print_history=print_history)
        valid_loss, valid_accuracy, inference_time = test(model, test_loader, criterion=criterion, device=device, mode='Validation', epoch=epoch, print_history=print_history)
        avg_inference_time += inference_time

        
        if(early_stopping):
            if early_stopper.early_stop(valid_loss):
                break

        train_loss_arr.append(train_loss)
        train_acc_arr.append(train_accuracy)

        try:
            if valid_accuracy>=hi_accuracy:
                hi_accuracy = valid_accuracy
                print(f'New best state saved with valid. acc. = {hi_accuracy} for epoch {epoch+1}.')
                print(f'Average inference time for a single validation batch: {inference_time}s.')
                model_state = model.state_dict()
        except:
            print("Cannot yet save model state dict.")

        validation_loss_arr.append(valid_loss)
        validation_acc_arr.append(valid_accuracy)


    if plot_history:
        plot_loss(train_loss=train_loss_arr, validation_loss=validation_loss_arr, title='CNN model loss')
        plot_acc(train_acc_arr, validation_acc_arr, title='CNN model accuracy')

    # model.load_state_dict(model_state)
    # test(model, test_loader, criterion=criterion, device=device, mode='Test', epoch=epoch)

    return hi_accuracy, avg_inference_time/num_epochs

In [6]:
def cv_split_testing_CNN(data, model_class, criterion, num_of_splits, activation_f, plot_history = True, print_history = True, early_stopping = True):

    from sklearn.model_selection import KFold

    splits = KFold(num_of_splits)

    cv_accuracy = []
    cv_inference_time = []
    for i, (train_split, test_split) in enumerate(splits.split(data)):
        print(f'Training and testing {i+1}/{num_of_splits} fold...')

        train_fold = data.loc[train_split,:]
        test_fold = data.loc[test_split,:]

        train_loader = create_loader_cv(train_fold, 64, False, True)
        test_loader = create_loader_cv(test_fold, 64, False, True)

        lr = 0.001
        model = model_class(activation_f=activation_f)
        optimizer = optim.RAdam(model.parameters(), lr=lr)
        
        fold_test_accuracy, inference_time = cv_CNN_model(train_loader, test_loader, model, criterion, optimizer, num_epochs=50, early_stopping=early_stopping, es_patience=5, es_delta=0.002, plot_history=plot_history, print_history=print_history)

        print(f'Highest validation accuracy in fold {i+1}/{num_of_splits}: {fold_test_accuracy}')

        # try:
        #     print(f'Avg. Inference time (a single forward pass): {model.inference_time/model.n_of_forward_passes}')
        # except:
        #     print('unknown inference time')

        cv_accuracy.append(fold_test_accuracy)
        cv_inference_time.append(inference_time)

    print(f'Average accuracy over {num_of_splits} folds: {np.mean(cv_accuracy)}')
    print(f'Average inference time over {num_of_splits} folds for a single validation batch: {np.mean(cv_inference_time)}')

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='none'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, input, target):
        BCE_loss = F.binary_cross_entropy(input, target, reduction='mean')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

#### Proposed CNN architectures

In [8]:
# define the CNN model
class CNN_team(nn.Module):
    def __init__(self, activation_f):
        super(CNN_team, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(64, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 1)
        self.bn0 = nn.BatchNorm1d(64)
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(32)
        self.bn4 = nn.BatchNorm1d(1)
        self.do = nn.Dropout(0.5)
        self.activation_f = activation_f

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.activation_f(self.conv1(x))
        x = self.pool(x)
        x = self.activation_f(self.conv2(x))
        x = self.pool(x)
        x = nn.AvgPool1d(kernel_size=x.shape[-1])(x)
        x = x.view(-1, 64)
        x = self.bn0(x)
        x = self.fc1(x)
        x = self.activation_f(x)
        x = self.bn1(x)
        x = self.do(x)
        x = self.fc2(x)
        x = self.activation_f(x)
        x = self.bn2(x)
        x = self.do(x)
        x = self.fc3(x)
        x = self.activation_f(x)
        x = self.bn3(x)
        x = self.do(x)
        x = torch.sigmoid(self.bn4(self.fc4(x)))
        return x

#### Example of usage for prediction

In [9]:
device = 'cuda'

# Load data from a CSV file
data = pd.read_csv('Teams_statistics.csv')
data.drop(['match_id','map','team_1','team_2'],axis=1, inplace=True)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
input_size = data.shape[1] - 1

In [ ]:
model_class = CNN_team
criterion = FocalLoss(alpha=0.95, gamma=2.6)
activation_f = nn.ReLU()


cv_split_testing_CNN(data, model_class, num_of_splits=5, criterion=criterion, activation_f=activation_f, plot_history=False, print_history=False, early_stopping=False)